1. 동별 행정구역 면적 데이터 추가
2. 다른 지자체 동 이름 중복 -> 서울시만 추출
3. 부암동, 종로5.6가동, 창신1동 등 동이름 중구난방 -> 숫자, 가, 동 제외하고 부암, 종로, 창신 등 으로 통일
4. 성북구/성북동, 용산구/용산1가동 과 같이 구, 동 이름 중복 -> 구, 동 구분, 동 추출
5. 면적 km^2 으로 통일

In [328]:
import numpy as np
import pandas as pd

# 전국 공원 데이터 CSV 파일 읽기
park_file_path = '../data/park.csv'

# UnicodeDecodeError 해결 - 한글 인코딩 utf-8 euc-kr cp949
park_df = pd.read_csv(park_file_path, encoding='euc-kr')

# 전국 인구 데이터 CSV 파일 읽기
population_file_path = '../data/population.csv'
population_df = pd.read_csv(population_file_path, encoding='euc-kr') 

In [329]:
# 공원 데이터 주소가 비어 있거나 NaN인 경우 제거
park_df = park_df[park_df['소재지지번주소'].notna() & (park_df['소재지지번주소'] != '')]

# 공원 데이터 서울특별시만 추출
park_df = park_df[park_df['소재지지번주소'].str.contains('서울특별시')]

# 주소에서 공백 구분한 리스트를 프레임으로 변경 후 '동'을 추출
park_df['법정동'] = park_df['소재지지번주소'].str.split(' ', expand=True)[2]

# 법정동 별 공원면적 합 데이터 프레임으로
filtered_park_df = park_df.groupby('법정동')['공원면적'].sum().reset_index()

print(filtered_park_df)

            법정동      공원면적
0         101-2    4551.6
1          12-1   16176.1
2        125-22   45170.5
3          14-3    5569.1
4          1499    3783.8
..          ...       ...
422         효창동  174662.4
423         후암동    2522.0
424         휘경동    9454.2
425         흑석동    4319.8
426  흑석동141-2일대  384026.0

[427 rows x 2 columns]


In [330]:
# 법정동 별 데이터 개수가 적은 것 같아 '다른 구 같은 동이름'처럼 중복값은 없는지 확인했으나 전부 False 나옴
#df = filtered_park_df.duplicated(['법정동'], keep='first')

In [331]:
# 인구 데이터 서울특별시만 추출
population_df = population_df[(population_df['시도명'] == '서울특별시')]

# 인구 데이터 법정동, 인구 컬럼만 추출
filtered_population_df = pd.DataFrame({
    '법정동': population_df['읍면동명'],
    '인구': population_df['계']
})

print(filtered_population_df)

     법정동     인구
0    청운동   2356
1    신교동   1369
2    궁정동    193
3    효자동    568
4    창성동    426
..   ...    ...
451  둔촌동  24235
452  암사동  66427
453  성내동  65506
454  천호동  85847
455  강일동  32866

[456 rows x 2 columns]


In [332]:
# 동별 인구 데이터와 공원 면적 데이터 인구 데이터 기준 병합
merged_df = pd.merge(filtered_park_df, filtered_population_df, left_on='법정동', right_on='법정동', how='right')

# 인구 당 공원 면적 계산
merged_df['인구 당 공원면적(m^2/명)'] = merged_df['공원면적'] / merged_df['인구']

print(merged_df)

     법정동      공원면적     인구  인구 당 공원면적(m^2/명)
0    청운동       NaN   2356               NaN
1    신교동       NaN   1369               NaN
2    궁정동       NaN    193               NaN
3    효자동       NaN    568               NaN
4    창성동       NaN    426               NaN
..   ...       ...    ...               ...
451  둔촌동  632733.0  24235         26.108232
452  암사동    1022.0  66427          0.015385
453  성내동       NaN  65506               NaN
454  천호동       NaN  85847               NaN
455  강일동  211206.1  32866          6.426279

[456 rows x 4 columns]


In [333]:
# 최종 결과 데이터 프레임 저장
output_file_path = '../data/park_result.csv'
merged_df.to_csv(output_file_path, encoding='utf-8')